<a href="https://colab.research.google.com/github/9-coding/Google_ML_Bootcamp_5th/blob/main/NL2SQL_unsloth_gemma2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install -U transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datetime import datetime
import torch

In [ ]:
use_gpu = torch.cuda.is_available()

In [ ]:
def extract_response(result):
    start_token = "<start_of_turn>model"
    end_token = "<end_of_turn>"

    # 시작점과 끝점 인덱스 찾기
    start_index = result.find(start_token)
    if start_index == -1:
        return None  # 시작 토큰이 없으면 None 반환
    start_index += len(start_token)

    end_index = result.find(end_token, start_index)
    if end_index == -1:
        return result[start_index:].strip()

    # 시작점과 끝점 사이의 텍스트 잘라내기
    return result[start_index:end_index].strip()

In [ ]:
def send_message(model, tokenizer, messages):
    global use_gpu

    formated_messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt")
    input_ids = tokenizer(formated_messages, return_tensors="pt")
    if use_gpu:
        input_ids = input_ids.to('cuda')

    start_time = datetime.now()
    print('generate start')

    outputs = model.generate(**input_ids, max_new_tokens=32, repetition_penalty=1.1)

    end_time = datetime.now()
    print('generate done')
    print('elapsed time:', (end_time - start_time).total_seconds())

    print(tokenizer.decode(outputs[0]) + '\n\n\n')
    return extract_response(tokenizer.decode(outputs[0]))

In [ ]:
from huggingface_hub import login
hf_token = 'TOKEN'

login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model = AutoModelForCausalLM.from_pretrained("9-coding/gemma-2-2b-it-nl2sql")
tokenizer = AutoTokenizer.from_pretrained("9-coding/gemma-2-2b-it-nl2sql", trust_remote_code=True)

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
if use_gpu:
    model.to('cuda')

In [ ]:
table_schema = 'CREATE TABLE person ( name VARCHAR, age INTEGER, address VARCHAR )'
user_qery = 'people whoes ages are older than 27 and name starts with k'

messages = [
    {
        "role": "user",
        "content": f"""Use the below SQL tables schemas paired with instruction that describes a task. make SQL query that appropriately completes the request for the provided tables. And make SQL query according the steps.
{table_schema}
step 1. check columns that user wants.
step 2. check condition that user wants.
step 3. make SQL query to get every information that user wants.

{user_qery}
"""
    }
]

In [ ]:
response = send_message(model, tokenizer, messages)
print('===============================================')
print(f'assistant: {response}')

generate start
generate done
elapsed time: 12.9594
<bos><bos><start_of_turn>user
Use the below SQL tables schemas paired with instruction that describes a task. make SQL query that appropriately completes the request for the provided tables. And make SQL query according the steps.
CREATE TABLE person ( name VARCHAR, age INTEGER, address VARCHAR )
step 1. check columns that user wants.
step 2. check condition that user wants.
step 3. make SQL query to get every information that user wants.

people whoes ages are older than 27 and name starts with k<end_of_turn>
<start_of_turn>model
SELECT name FROM person WHERE age > 27 AND name LIKE "k%"<end_of_turn>



assistant: SELECT name FROM person WHERE age > 27 AND name LIKE "k%"
